In [2]:
import sys
print(sys.executable)

/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/bin/python3


In [4]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'\nDevice: {device}')


Device: cuda:0


#
##  codegen-350M-multi

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-multi")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-multi", device_map='auto')

print(model)

CodeGenForCausalLM(
  (transformer): CodeGenModel(
    (wte): Embedding(51200, 1024)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-19): 20 x CodeGenBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): CodeGenAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (qkv_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=False)
        )
        (mlp): CodeGenMLP(
          (fc_in): Linear(in_features=1024, out_features=4096, bias=True)
          (fc_out): Linear(in_features=4096, out_features=1024, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=51200, bias=True)
)


In [4]:
def print_modules(model, prefix=''):
    for name, module in model.named_children():
        print(f"{prefix}{name}")
        if len(list(module.children())) > 0:
            print_modules(module, f"{prefix}{name}.")
        
# Print all the modules in your model
print_modules(model)

transformer
transformer.wte
transformer.drop
transformer.h
transformer.h.0
transformer.h.0.ln_1
transformer.h.0.attn
transformer.h.0.attn.attn_dropout
transformer.h.0.attn.resid_dropout
transformer.h.0.attn.qkv_proj
transformer.h.0.attn.out_proj
transformer.h.0.mlp
transformer.h.0.mlp.fc_in
transformer.h.0.mlp.fc_out
transformer.h.0.mlp.act
transformer.h.0.mlp.dropout
transformer.h.1
transformer.h.1.ln_1
transformer.h.1.attn
transformer.h.1.attn.attn_dropout
transformer.h.1.attn.resid_dropout
transformer.h.1.attn.qkv_proj
transformer.h.1.attn.out_proj
transformer.h.1.mlp
transformer.h.1.mlp.fc_in
transformer.h.1.mlp.fc_out
transformer.h.1.mlp.act
transformer.h.1.mlp.dropout
transformer.h.2
transformer.h.2.ln_1
transformer.h.2.attn
transformer.h.2.attn.attn_dropout
transformer.h.2.attn.resid_dropout
transformer.h.2.attn.qkv_proj
transformer.h.2.attn.out_proj
transformer.h.2.mlp
transformer.h.2.mlp.fc_in
transformer.h.2.mlp.fc_out
transformer.h.2.mlp.act
transformer.h.2.mlp.dropout
trans

In [6]:
## candidate prompt format

prompt = """# write a SPARQL query for:
# what is the population of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# write a SPARQL query for:
# what is the population of Italy?

# http://www.w3.org/TR/rdf-sparql-query/#query-query-population-of-Italy

# http://www.w3.org/TR/rdf-sparql-query/#query-query-population-of-Italy

# http://www.w3.org/TR/rdf-sparql-query/#query-query-population-of-Italy

# http://


#
## Data 

In [7]:
import pandas as pd

df = pd.read_csv("data/nspm-fine-tuning/train.csv")

# df = df[['prompt', 'completion']]
df

,Unnamed: 0,prompt,completion
0,3382376,# write a SPARQL query for:\n# How many ICD9 d...,select count(*) as ?x where{dbr:Friedreich's_a...
1,2559914,# write a SPARQL query for:\n# What was the fa...,select ?x where{dbr:Destrehan_High_School dbo:...
2,2990831,# write a SPARQL query for:\n# Does adventist ...,ask where{dbr:Adventist_Girls_High_School dbo:...
3,1637967,# write a SPARQL query for:\n# Did aliens area...,ask where{dbr:Aliens_Area dbo:lastPublicationD...
4,1507546,# write a SPARQL query for:\n# Did bridget jon...,ask where{dbr:Bridget_Jones:_The_Edge_of_Reaso...
...,...,...,...
99995,7260453,# write a SPARQL query for:\n# What is the pop...,select ?x where{dbr:Southern_Yukaghir_language...
99996,6784536,# write a SPARQL query for:\n# Is washington c...,"ask where{dbr:Mitchell_County,_North_Carolina ..."
99997,3733762,# write a SPARQL query for:\n# How many marrie...,select count(*) as ?x where{dbr:Bernardine_Doh...
99998,7637742,# write a SPARQL query for:\n# What are the ba...,select ?x where{dbr:Battle_of_Magdala dbo:resu...


In [8]:
for index, row in df.sample(n=5).iterrows():
    print(f"Prompt:\n{row['prompt']}\n\nSPARQL:\n{row['completion']}\n{'='*80}")

Prompt:
# write a SPARQL query for:
# When will be 185th paratroopers artillery regiment "folgore" anniversary ?

SPARQL:
select ?x where{dbr:185th_Paratroopers_Artillery_Regiment_%22Folgore%22 dbo:anniversary ?x . <B> dbo:anniversary ?x }
Prompt:
# write a SPARQL query for:
# Who invented the the duchess of malfi (local language) ?

SPARQL:
select ?x where{dbr:The_Duchess_of_Malfi dbo:originalLanguage ?x }
Prompt:
# write a SPARQL query for:
# Did erode–tiruchirappalli line actually have 24 stations ?

SPARQL:
select count(*) as ?x where{dbr:Erode–Tiruchirappalli_line dbo:numberOfStations ?x }
Prompt:
# write a SPARQL query for:
# Did lev pontryagin have doctoral advisor ?

SPARQL:
ask where{dbr:Lev_Pontryagin dbo:doctoralAdvisor ?x }
Prompt:
# write a SPARQL query for:
# What is the tuition of saint francis central coast catholic high school High School ?

SPARQL:
select count(*) as ?x where{dbr:Saint_Francis_Central_Coast_Catholic_High_School dbo:tuition ?x }


#
## Training for fine-tuning

In [1]:
%%time
!python finetune-codegen-350M-multi.py\
--model_path="Salesforce/codegen-350M-multi"\
--output_dir="./codegen-checkpoints"\
--seq_length=512\
--batch_size=4\
--eos_token_id=50256\
--save_freq=2000\
--seed=42




Using CUDA device: NVIDIA GeForce RTX 3090
Memory Allocated: 0.00 MB
Memory Reserved: 0.00 MB



Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-3615beb83b6f428b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|████████████████████████████████████████████| 2/2 [00:00<00:00, 970.01it/s]
Size of the train set: 100000. Size of the validation set: 20000
100%|███████████████████████████████████████| 400/400 [00:00<00:00, 5996.49it/s]
The character to token ratio of the dataset is: 2.95
Loading the model
/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/lib/python3.11/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 1966080 || all params: 358678528 || trainable%: 0.5481454412570802
Starting main loop
Training...
/home/mehr-shahin/repos/dbp/neu

## 2nd round of fine-tuning

In [4]:
import os

num_cores = os.cpu_count()
print(num_cores)


48


In [1]:
%%time
!python finetune-codegen-350M-multi.py\
--model_path="Salesforce/codegen-350M-multi"\
--output_dir="./codegen-checkpoints/2nd_round"\
--seq_length=512\
--batch_size=16\
--eos_token_id=50256\
--save_freq=2000\
--seed=42\
--num_workers=40





Using CUDA device: NVIDIA GeForce RTX 3090



Found cached dataset csv (/home/mehr-shahin/.cache/huggingface/datasets/csv/default-3615beb83b6f428b/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 1019.15it/s]
Size of the train set: 100000. Size of the validation set: 20000
100%|███████████████████████████████████████| 400/400 [00:00<00:00, 6264.76it/s]
The character to token ratio of the dataset is: 2.95
Loading the model
/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/lib/python3.11/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
trainable params: 1966080 || all params: 358678528 || trainable%: 0.5481454412570802
Starting main loop
Training...
/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/starcoder-1b/lib/python3.11/sit

In [2]:
%%time
!python merge_peft_adapters.py\
--base_model_name_or_path="Salesforce/codegen-350M-multi"\
--peft_model_path="./codegen-checkpoints/2nd_round/final_checkpoint/"\
--merged_model_name_suffix="nspm"

Model saved to Salesforce/codegen-350M-multi-nspm
CPU times: user 79.1 ms, sys: 13.5 ms, total: 92.6 ms
Wall time: 12.1 s



### Load fine-tuned checkpoints


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint="Salesforce/codegen-350M-multi-nspm"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, use_auth_token=True).to(device)


In [6]:
## candidate prompt format

prompt = """# write a SPARQL query for:
# what is the population of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# write a SPARQL query for:
# what is the population of Italy?

Answer: select?x where{dbr:Italy dbo:population?x }<|endoftext|>


In [6]:
## candidate prompt format

prompt = """# write a SPARQL query for:
# what is the population of Italy?
"""

inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# write a SPARQL query for:
# what is the population of Italy?

Answer: select?x where{dbr:Italy_(horse) dbo:populationPlace?x }<|endoftext|>


In [7]:
## candidate prompt format

prompt = """# write a SPARQL query for:
# What is the foundation of diocese of birobidzhan ?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# write a SPARQL query for:
# What is the foundation of diocese of birobidzhan?

Answer: select?x where{dbr:Birobidzhan dbo:foundation?x }<|endoftext|>


In [7]:
## candidate prompt format

prompt = """# write a SPARQL query for:
# What is the foundation of diocese of birobidzhan ?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# write a SPARQL query for:
# What is the foundation of diocese of birobidzhan?

Answer: select?x where{dbr:Birobidzhan dbo:foundation?x }<|endoftext|>


In [10]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What was the population of Italy in 2000 ?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



# write a SPARQL query for:
# What was the population of Italy in 2000?

Answer: select?x where{dbr:Italy_in_2000 dbo:population?x }<|endoftext|>


In [8]:
## candidate prompt format

prompt = """
# write a SPARQL query for:
# What was the population of Italy in 2000 ?
"""


inputs = tokenizer.encode_plus(prompt, return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



# write a SPARQL query for:
# What was the population of Italy in 2000?

Answer: select?x where{dbr:Italy_in_2000 dbo:populationUrban?x }<|endoftext|>


# 
## Evaluate fine-tuned model (40% training progress) 

# 

### Calculate BELUE score

In [29]:
%%time

!python evaluate_model.py\
--model_name_or_path="Salesforce/codegen-350M-multi-merged"\
--data_path="./data/nspm-fine-tuning/test.csv"\
--metric_name="bleu"\
--report_file_name="eval_results_codegen350M_multi_01"\
--verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:root:Loading tokenizer and model on cuda:0.

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp9mt3leu1
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp9mt3leu1/_remote_module_non_scriptable.py

Calculating scores with <bleu> as evaluation metric
...


/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/evaluate_model.py:53: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric(metric_name)
INFO:root:Processed 0 rows. BLEU Score - Generated: 0.0, Pos

# 
## Evaluate fine-tuned model (100% training progress) 

# 

### Calculate BELUE score

In [9]:
%%time

!python evaluate_model.py\
--model_name_or_path="Salesforce/codegen-350M-multi-nspm"\
--data_path="./data/nspm-fine-tuning/test.csv"\
--metric_name="bleu"\
--report_file_name="eval_results_codegen350M_multi_nspm"\
--verbose

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:root:Loading tokenizer and model on cuda:0.

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /tmp/tmp6bym6kah
INFO:torch.distributed.nn.jit.instantiator:Writing /tmp/tmp6bym6kah/_remote_module_non_scriptable.py

Calculating scores with <bleu> as evaluation metric
...


/home/mehr-shahin/repos/dbp/neural-qa/gsoc/mehrzad/evaluate_model.py:53: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric(metric_name)
INFO:root:Processed 0 rows. BLEU Score - Generated: 0.0, Pos

In [10]:
import pandas as pd

df = pd.read_csv("eval_results_codegen350M_multi_nspm.csv")
df
                     

,Prompt,Actual SPARQL,Generated SPARQL,BLEU Score (Generated),Post-Processed SPARQL,BLEU Score (Post-Processed)
0,# write a SPARQL query for:\n# What is the Lin...,select ?x where{dbr:Karl_Marx dbo:birthPlace ?...,\nAnswer: select?x where{dbr:Karl_Marx dbo:bir...,0.0,select ?x where{dbr:Karl_Marx dbo:birthPlace ?...,0.707107
1,# write a SPARQL query for:\n# Is diocesan col...,ask where{dbr:Morgan_Newman dbo:school dbr:Dio...,\nAnswer: ask where{dbr:Morgan_Newman dbo:coll...,0.0,ask where{dbr:Morgan_Newman dbo:collegeSchool ...,0.000000
2,# write a SPARQL query for:\n# What is the cou...,select ?x where{dbr:Canário dbo:birthPlace ?x1...,\nAnswer: select?x where{dbr:Canário dbo:birth...,0.0,select ?x where{dbr:Canário dbo:birthPlace ?x1...,1.000000
3,# write a SPARQL query for:\n# What is the por...,select ?x where{dbr:Stella_Bonasera dbo:portra...,\nAnswer: select?x where{dbr:Lucky_Simpson dbo...,0.0,select ?x where{dbr:Lucky_Simpson dbo:portraye...,0.000000
4,# write a SPARQL query for:\n# Has south afric...,ask where{dbr:South_African_Class_16E dbo:rebu...,\nAnswer: ask where{dbr:South_African_Class_16...,0.0,ask where{dbr:South_African_Class_16e dbo:rebu...,0.000000
...,...,...,...,...,...,...
19995,# write a SPARQL query for:\n# How many diamet...,select count(*) as ?x where{dbr:Volterra_(crat...,\nAnswer: select count(*) as?x where{dbr:Volte...,0.0,select count(*) as ?x where{dbr:Volterra dbo:d...,0.500000
19996,# write a SPARQL query for:\n# How is hyatt re...,select ?x where{dbr:Hyatt_Regency_Chicago dbo:...,\nAnswer: select?x where{dbr:Hyatt_Regency_Chi...,0.0,select ?x where{dbr:Hyatt_Regency_Chicago dbo:...,0.658037
19997,# write a SPARQL query for:\n# Who is thomas d...,select ?x where{dbr:Thomas_Dudley dbo:father ?...,\nAnswer: select?x where{dbr:Thomas_Dudley dbo...,0.0,select ?x where{dbr:Thomas_Dudley dbo:father ?...,1.000000
19998,# write a SPARQL query for:\n# What are most o...,select count(*) as ?x where{dbr:Schuylkill_Ars...,\nAnswer: select?x where{dbr:Schuylkill_Arsena...,0.0,select ?x where{dbr:Schuylkill_Arsenal_railroa...,0.000000


In [11]:
print("\n")
print("Descriptive Statistics for BLEU Scores (Generated):")
print(df['BLEU Score (Generated)'].describe())
print("\n")
print("Descriptive Statistics for BLEU Scores (Post-Processed):")
print(df['BLEU Score (Post-Processed)'].describe())



Descriptive Statistics for BLEU Scores (Generated):
count    20000.000000
mean         0.015202
std          0.085129
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          0.680375
Name: BLEU Score (Generated), dtype: float64


Descriptive Statistics for BLEU Scores (Post-Processed):
count    20000.000000
mean         0.431073
std          0.441416
min          0.000000
25%          0.000000
50%          0.379918
75%          1.000000
max          1.000000
Name: BLEU Score (Post-Processed), dtype: float64


In [92]:
# Filter rows where 'BLEU Score (Post-Processed)' is 0
zero_score_df = df[df['BLEU Score (Post-Processed)'] == 0]

# error analysis based on sample rows where BLEU Score (Post-Processed) is 0
sampled_df = zero_score_df.sample(n=min(100, len(zero_score_df)))

for index, row in sampled_df[['Prompt','Actual SPARQL', 'Post-Processed SPARQL']].iterrows():
    print("Prompt:    ", row["Prompt"][28:], "\n")
    print("Actual:    ", row["Actual SPARQL"])
    print("Processed: ", row["Post-Processed SPARQL"])
    print("-"*100)  

Prompt:     # Is the emerald number of parking spaces of claridge icon ? 

Actual:     ask where{dbr:Claridge_Icon dbo:numberOfParkingSpaces dbr:The_Emerald_(building) }
Processed:  ask where{dbr:Claridge_Icon dbo:numberOfParkingSpaces dbr:Emerald }
----------------------------------------------------------------------------------------------------
Prompt:     # Did the george houston house have the NRHP reference number ? 

Actual:     ask where{dbr:George_Houston_House dbo:nrhpReferenceNumber ?x }
Processed:  ask where{dbr:The_George_Houston_House dbo:nrhpReferenceNumber ?x }
----------------------------------------------------------------------------------------------------
Prompt:     # How should I know the ingredient names for mache ? 

Actual:     select ?x where{dbr:Mache_(food) dbo:ingredientName ?x }
Processed:  select ?x where{dbr:Mache dbo:ingredientName ?x }
----------------------------------------------------------------------------------------------------
Prompt:     # W

In [90]:
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(f'\nDevice: {device}')


Device: cuda:0


In [91]:
from transformers import AutoModelForCausalLM, AutoTokenizer

checkpoint="bigcode/starcoderbase-1b-merged"

tokenizer = AutoTokenizer.from_pretrained(checkpoint, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(checkpoint, use_auth_token=True).to(device)


In [23]:
## candidate prompt format

prompt = "Question: # write a SPARQL query for:\n# What are cadolzburg area code ?"


inputs = tokenizer.encode_plus(prompt.strip(), return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: # write a SPARQL query for:
# What are cadolzburg area code?

Answer: select?x where{dbr:Cadolzburg dbo:areaCode?x }<|endoftext|>


In [24]:
## candidate prompt format

prompt = "Question: # write a SPARQL query for:\n# How many province did avola grow in ?"


inputs = tokenizer.encode_plus(prompt.strip(), return_tensors="pt").to(device)
outputs = model.generate(inputs['input_ids'], max_new_tokens=100)
print(tokenizer.decode(outputs[0]))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


Question: # write a SPARQL query for:
# How many province did avola grow in?

Answer: select count(*) as?x where{dbr:Avola dbo:province?x }<|endoftext|>


In [16]:
import pandas as pd
df_test = pd.read_csv("data/nspm-fine-tuning/test.csv")

df_sample = df_test.sample(n=10, random_state=0)

for i in df_sample['prompt'].to_list():
    print(i, "\n")

# write a SPARQL query for:
# What's the percentage of area water in bert cremean's Death place ? 

# write a SPARQL query for:
# Did calabritto have saint ? 

# write a SPARQL query for:
# How many ideologies did ganatantri dal have ? 

# write a SPARQL query for:
# Who is the second driver of 1976 race of champions ? 

# write a SPARQL query for:
# What is the ship beam of voyager of the seas as well as uss fort worth ? 

# write a SPARQL query for:
# How many province did avola grow in ? 

# write a SPARQL query for:
# What are cadolzburg area code ? 

# write a SPARQL query for:
# What ISO code for catalan language ? 

# write a SPARQL query for:
# What is the population of the city of sohawa tehsil ? 

# write a SPARQL query for:
# What is the revenue of zale corporation ? 



In [ ]:
select?x where{dbr:Bert_Cremean dbo:deathPlace?x1.?x1 dbo:percentageOfAreaWater?x }<|endoftext|>

In [21]:
for i in df_sample['completion'].to_list():
    print(i, "\n")

select ?x where{dbr:Bert_Cremean dbo:deathPlace ?x1 . ?x1 dbo:percentageOfAreaWater ?x } 

ask where{dbr:Calabritto dbo:saint ?x } 

select count(*) as ?x where{dbr:Ganatantri_Dal dbo:ideology ?x } 

select ?x where{dbr:1976_Race_of_Champions dbo:secondDriver ?x } 

select ?x where{dbr:Voyager_of_the_Seas dbo:shipBeam ?x . dbr:USS_Fort_Worth dbo:shipBeam ?x } 

select count(*) as ?x where{dbr:Avola dbo:province ?x } 

select count(*) as ?x where{dbr:Cadolzburg dbo:areaCode ?x } 

select count(*) as ?x where{dbr:Catalan_language dbo:iso6391Code ?x } 

select ?x where{dbr:Sohawa_Tehsil dbo:populationRural ?x } 

select count(*) as ?x where{dbr:Zale_Corporation dbo:revenue ?x } 

